### Acte 4 : Une Trente-septième vue du Mont Fuji ?

In [ ]:
from math import *
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D

### Visualisation 3D et 2D des données topographiques du Mont Fuji

- `X` et `Y` définissent un quadrillage régulier de 100×100 points sur la zone autour du sommet, couvrant [-4,4] km en abscisse et ordonnée.  
- La matrice `Z` contient les altitudes récupérées depuis le fichier `satellite.txt` pour chaque point du quadrillage.  
- Le graphique 3D (`ax.plot_surface`) affiche la surface du relief du Mont Fuji avec une palette rose, permettant d’observer la forme générale de la montagne.  
- Le graphique 2D de contours (`plt.contour`) représente les courbes de niveau du relief, offrant une vue en plan des altitudes.

Ces visualisations sont une première étape pour analyser la topographie et préparer la navigation vers le sommet en utilisant le gradient du relief.

In [ ]:
X = np.linspace(-4, 4, 100)
Y = np.linspace(-4, 4, 100)
X, Y = np.meshgrid(X, Y)

fichier = open("satellite.txt",'r')
lignes = fichier.readlines()
taille=len(lignes)

Z = np.loadtxt("satellite.txt")

fig, ax = plt.subplots(subplot_kw = {"projection": "3d"})
ax.plot_surface(X, Y, Z, cmap = "pink")
plt.show()
plt.contour(X, Y, Z, cmap = "pink")

### Calcul du gradient et recherche du point voisin sur la grille

- `alpha = 1e-4` : taille du pas pour la progression sur la montagne.
- `Z_grad_x` et `Z_grad_y` sont les matrices des dérivées partielles de l’altitude `Z` selon `x` et `y`, calculées avec `np.gradient`. Elles représentent le gradient du relief.

- La fonction `trouver_tildes(X, Y, x, y)` permet de trouver dans la grille le point `(X[~,~], Y[~,~])` le plus proche d’une position réelle `(x, y)` donnée (qui peut ne pas correspondre exactement à un point de la grille).

- Elle parcourt tous les points du quadrillage, calcule la distance au point `(x, y)`, et retourne les indices du point ayant la plus petite distance.

Cette fonction est essentielle pour appliquer la règle de déplacement par gradient même lorsque la position calculée ne correspond pas exactement à un point du quadrillage.

In [1]:
alpha = 1e-4
Z_grad_x, Z_grad_y = np.gradient(Z)

def trouver_tildes(X, Y, x, y):
    min_dist = float('inf')
    min_x, min_y = 0, 0

    for i in range(taille):
        for j in range(taille):
            dx = X[i, j] - x
            dy = Y[i, j] - y
            dist = dx*dx + dy*dy

            if dist < min_dist:
                min_dist = dist
                min_x, min_y = j, i #on inverse car i c'est les y et j les x

    return min_x, min_y

NameError: name 'np' is not defined

### Simulation du chemin de montée vers le sommet du Fuji par descente de gradient

- `trajetx` et `trajety` enregistrent les coordonnées successives du parcours.
- Le point de départ est fixé à `(x0, y0) = (3.5, 0)`.
- `maxZ` correspond à l’altitude maximale du Fuji (3776 m).
- À chaque étape :
  - On ajoute la position actuelle aux listes de trajet.
  - On détermine l’indice `(i, j)` du point du quadrillage le plus proche de `(x0, y0)` grâce à la fonction `trouver_tildes`.
  - On récupère le gradient de l’altitude en ce point (`grad_x`, `grad_y`).
  - On fait un pas dans la direction du gradient (montée de la pente) en ajustant `(x0, y0)` selon `alpha`.
  - On vérifie si l’altitude `Z[i, j]` dépasse la hauteur du sommet (`maxZ`). Si oui, la boucle s’arrête et affiche le nombre de pas nécessaires.

Ce code simule donc pas à pas l’ascension vers le sommet en suivant la direction de la pente maximale, jusqu’à atteindre ou dépasser l’altitude du Mont Fuji.


In [ ]:
trajetx=[]
trajety=[]
trajetz=[]
x0,y0 = 3.5,0
depart = (x0,y0,Z[trouver_tildes(X, Y, x0, y0)])
maxZ=3776
for step in range(1,1000):

    i,j = trouver_tildes(X, Y, x0, y0)

    trajetx.append(x0)
    trajety.append(y0)
    trajetz.append(Z[i,j])

    grad_x = Z_grad_x[i, j]
    grad_y = Z_grad_y[i, j]

    x0 = x0 + alpha * grad_x
    y0 = y0 + alpha * grad_y

    if maxZ < Z[i,j]:
        print("\n" + "="*50)
        print("🏔️  Félicitations, vous avez atteint le sommet du Fuji ! 🏔️")
        print(f"Nombre de pas nécessaires : {step}")
        print("Vous voilà au sommet, là où Hokusai a peut-être laissé son ultime estampe...")
        print("="*50 + "\n")
        break

### Visualisation du trajet vers le sommet

####  Vue de haut (2D)

Pour avoir une première représentation claire de l’ascension, on utilise une **vue en plongée** du Mont Fuji.  
Les lignes de niveau bleues permettent de visualiser les variations d’altitude comme sur une carte topographique.  
Sur cette carte, le **trajet suivi** par l’algorithme du gradient est tracé en **rouge pointillé**.  
Cela permet de voir comment l’itinéraire serpente progressivement vers le sommet, en suivant toujours la direction de la plus grande pente locale.

---

#### Vue 3D (relief en perspective)

La seconde visualisation donne une **représentation en relief** du Mont Fuji.  
On y voit le **volume de la montagne** avec ses pentes, ses crêtes et son sommet.  
Le **trajet d’ascension** est représenté en rouge sur la surface, rendant très lisible l’évolution du chemin dans l’espace.  
Le **point de départ** est marqué par une **étoile rouge** et annoté d’un petit texte *"Départ"*, comme sur une carte de randonnée.  

In [2]:
#Vue de haut
# Affichage de la surface du Fuji
plt.contour(X, Y, Z, cmap = "Blues")

# Ajout de la trajectoire (en rouge)
plt.plot(trajetx,trajety,'r--')

# Ajout axe et titre
plt.xlabel('X (km)')
plt.ylabel('Y (km)')
plt.title("Ascension simulée du Mont Fuji par gradient (Vue de Haut)")
plt.show()

#3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Affichage de la surface du Fuji
ax.plot_surface(X, Y, Z, cmap='Blues', alpha=0.6, edgecolor='none')

# Ajout de la trajectoire en 3D (en rouge)
ax.plot(trajetx, trajety, trajetz, color='red', linewidth=3, linestyle='--', markersize=5, label='Trajet vers le sommet')

# Point de départ
ax.scatter(depart[0], depart[1], depart[2], color='red', s=100, marker='*', label='Départ')
ax.text(depart[0], depart[1]+0.3, depart[2] + 0.3, "Départ", color='red', fontsize=12)

# Ajout axe et titre
ax.set_xlabel('X (km)')
ax.set_ylabel('Y (km)')
ax.set_zlabel('Altitude (m)')
ax.set_title("Ascension simulée du Mont Fuji par gradient (Vue 3D)")
ax.legend()

plt.show()

NameError: name 'plt' is not defined